In [1]:
import numpy as np
import pandas as pd

In [2]:
variables = np.array(
    [
        "a","b","c","d","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u",
        "v","w","x","y","z"
    ]
)
digits = np.array(["0","1","2","3","4","5","6","7","8","9"])
arithmetic = np.array(["-","+","*","/"])
equalities = ["==", ">", "<", ">=", "<="]
power = "**"
assignment = "="
if_start = "if CONDITION:\n"
if_else = "else:\n"
if_elif = "elif CONDITION:\n"
list_comprehension = "[FUNCTION(VARIABLE) for VARIABLE in LIST]"
function_start = "def NAME(ARGUMENTS):\n"
function_return = "return(NAME)"

Expression: variable = many digits or arithmetic or power or variable

In [3]:
def make_digit():
    s = "".join(
        np.random.choice(digits, size=np.random.choice(np.arange(1,5)))
    )
    if s[0] == "0" and len(s) > 1:
        s = s[1:]
    return(s)

In [4]:
def init_variable(current_variables):
    if np.random.choice([True, False]):
        s = np.random.choice(variables)+" = "+random_expression()
    else:
        if len(current_variables) > 0:
            s = np.random.choice(current_variables)+" = "+random_expression()
        else:
            s = np.random.choice(variables)+" = "+random_expression()
    return(s)

In [5]:
def random_expression():
    s = make_digit()
    while np.random.choice([True, False]):
        if np.random.choice([True, False]):
            s += np.random.choice(arithmetic)
            s += make_digit()
    return(s)

In [7]:
def make_condition(current_variables):
    if len(current_variables) > 0:
        var = np.random.choice(current_variables)
    else:
        var = np.random.choice(variables)
    s = var + " " + np.random.choice(equalities) + " "
    what = np.random.choice(["variable", "digit", "expression"])
    if what == "variable":
        if len(current_variables):
            s += np.random.choice(current_variables)
        else:
            s += np.random.choice(variables)
    elif what == "digit":
        s += make_digit()
    else:
        s += random_expression()
    return(s)

In [8]:
n = np.random.choice(np.arange(5,10))

In [9]:
def make_code(n):
    code = []
    current_variables = []
    open_ifs = []
    block_count = 0
    prev = "start"
    for a in range(n):
        what = np.random.choice(
            [
                "assignment", "if_start", "if_elif", "if_else"#, "b++", "b--"
            ], p = [0.5, 0.3, 0.1, 0.1]
        )
        code.append(block_count*"    ")
        if what == "assignment":
            line = init_variable(current_variables)
            current_variables.append(line[0])
            current_variables = list(set(current_variables))
            code.append(line)
        elif what == "if_start":
            if a != n-1 and n != 0:
                code.append(if_start.replace("CONDITION", make_condition(current_variables)))
                block_count += 1
                open_ifs.append(block_count)
            else:
                line = init_variable(current_variables)
                current_variables.append(line[0])
                current_variables = list(set(current_variables))
                code.append(line)
        elif what == "if_elif":
            if prev == "assignment" and a != n-1 and n != 0:
                if len(open_ifs) > 0:
                    if open_ifs[-1] == block_count:
                        code.append(
                            if_elif.replace("CONDITION", make_condition(current_variables))
                        )
                        block_count += 1
            else:
                line = init_variable(current_variables)
                current_variables.append(line[0])
                current_variables = list(set(current_variables))
                code.append(line)
        elif what == "if_elif":
            if prev == "assignment" and a != n-1 and n != 0:
                if len(open_ifs) > 0:
                    if open_ifs[-1] == block_count:
                        code.append(
                            if_else.replace("CONDITION", make_condition(current_variables))
                        )
                        block_count += 1
            else:
                line = init_variable(current_variables)
                current_variables.append(line[0])
                current_variables = list(set(current_variables))
                code.append(line)
        code += "\n"
        prev = what
      #  print(a, what)
    if len(current_variables) > 0:
        to_output = np.random.choice(current_variables)
    else:
        to_output = np.random.choice(variables)
    code += "print("+np.random.choice(current_variables)+")"
    return(code, current_variables)

In [16]:
import re
import os
import subprocess
import os.path as op
from tqdm import tqdm

In [11]:
if not op.exists("tasks"):
    os.makedirs("tasks")

In [119]:
if not op.exists("pdfs"):
    os.makedirs("pdf")

In [150]:
if not op.exists("tex"):
    os.makedirs("tex")

In [12]:
if not op.exists("answers"):
    os.makedirs("answers")

In [13]:
if not op.exists("variables"):
    os.makedirs("variables")

In [60]:
N_tasks = 400

In [61]:
for a in range(N_tasks):
    fn = op.join("tasks", str(a)+".py")
    ans_fn = op.join("answers", str(a)+".txt")
    var_fn = op.join("variables", str(a)+".txt")
    code, vs = make_code(np.random.choice(np.arange(5,20)))
    with open(fn, "w") as oh:
        oh.write("".join(code))
    the_code = subprocess.Popen(
        "python "+fn, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True
    )
    stdout, stderr = the_code.communicate()
    with open(ans_fn, "w") as oh:
        if len(stdout) != 0:
            print(stdout)
            oh.write("correct\n")
            oh.write(stdout.decode())
        else:
            oh.write("error\n")
            oh.write(stderr.decode())
    with open(var_fn, "w") as oh:
        oh.write(",".join(vs))

b'4474\n'
b'6134\n'
b'8\n'
b'74\n'
b'5530\n'
b'-29\n'
b'0.6949152542372882\n'
b'0\n'
b'958\n'
b'15\n'
b'7807\n'
b'21\n'
b'6063.0\n'
b'3\n'
b'0\n'
b'864\n'
b'8\n'
b'4.919762258543834\n'
b'43.5\n'
b'60\n'
b'5\n'
b'9\n'
b'23\n'
b'6770\n'
b'7917224\n'
b'904\n'
b'9354\n'
b'-150350\n'
b'7\n'
b'22\n'
b'6\n'
b'0.004324683965402528\n'
b'1377\n'
b'0\n'
b'4315\n'
b'7258\n'
b'90642\n'
b'3192\n'
b'1\n'
b'3728\n'
b'5367\n'
b'53\n'
b'510\n'
b'617\n'
b'615\n'
b'28\n'
b'9\n'
b'871\n'
b'6012\n'
b'8913\n'
b'6\n'
b'139\n'
b'7\n'
b'32\n'
b'292\n'
b'8022\n'
b'80\n'
b'20\n'
b'45\n'
b'118\n'
b'69\n'
b'5236\n'
b'142\n'
b'866\n'
b'4428\n'
b'6496\n'
b'8330\n'
b'673\n'
b'16\n'
b'685\n'
b'22355\n'
b'88\n'
b'6.857142857142857\n'
b'746\n'
b'4946\n'
b'6\n'
b'331\n'
b'937\n'
b'5\n'
b'862\n'
b'6\n'
b'-470\n'
b'-67340.5\n'
b'8679\n'
b'3327\n'
b'431\n'
b'4\n'
b'1.6\n'
b'957\n'
b'0.0013706140350877192\n'
b'5809\n'
b'8\n'
b'-7.604875702870672\n'
b'731\n'
b'79\n'
b'9774\n'
b'89\n'
b'5452\n'
b'95\n'
b'504\n'
b'0\n'
b'43\n'
b

In [73]:
def form_answers(n):
    ans_fn = op.join("answers", str(n)+".txt")
    with open(ans_fn, "r") as ih:
        last_line = ih.readlines()[-1]
    return(last_line)

In [107]:
def form_incorrect_answers(n, corrects):
    var_fn = op.join("variables", str(n)+".txt")
    with open(var_fn, "r") as ih1:
        chosen = np.random.choice(corrects)
        variables_in_file = ih1.read().replace("\n", "").split(",")
        what = np.random.choice(["chosen", "random"])
        if what == "chosen":
            if "name" in chosen:
                if len(variables_in_file)>1:
                    rpl = np.random.choice(variables_in_file)
                    chosen = re.sub("name \'[a-z]\'", "name \'"+rpl+"\'", chosen)
        elif what == "random":
            chosen = make_digit()
    return(chosen)

In [108]:
form_answers(6)

"NameError: name 'o' is not defined\n"

In [111]:
form_incorrect_answers(6, correct_answers)

"NameError: name 'z' is not defined\n"

In [75]:
correct_answers = [form_answers(a) for a in np.arange(N_tasks)]

In [114]:
incorrect_answers_1 = [form_incorrect_answers(a, correct_answers) for a in np.arange(N_tasks)]
incorrect_answers_2 = [form_incorrect_answers(a, correct_answers) for a in np.arange(N_tasks)]
incorrect_answers_3 = [form_incorrect_answers(a, correct_answers) for a in np.arange(N_tasks)]

In [160]:
with open("test_form.tex", "r") as ih:
    form = ih.read()

In [123]:
variant = np.random.choice(np.arange(N_tasks), 10)

In [124]:
variant

array([124, 343, 133,  78,  23, 208,  85, 125,  23, 383])

In [132]:
from copy import deepcopy

In [152]:
def get_variant():
    variant = np.random.choice(np.arange(N_tasks), 10)
    current_form = deepcopy(form)
    for i,a in enumerate(variant):
        correct_answer = form_answers(a)
        incorrects = [form_incorrect_answers(a, correct_answers) for b in range(3)]
        anssss = np.random.permutation(incorrects + [correct_answer])
        question_number = "Q"+str(i)
        current_form = current_form.replace(question_number, str(a))
        answer_number_base = "A"+str(i)
        for j,b in enumerate(anssss):
            current_form = current_form.replace(answer_number_base+str(j+1), b)
        filename = "test.py}%"+str(i)
        current_form = current_form.replace(filename, op.join("../tasks", str(a)+".py}"))
    return(current_form)

In [161]:
for a in tqdm(range(30)):
    with open(op.join("tex/", str(a)+".tex"), "w") as oh:
        cf = get_variant()
        oh.write(cf.replace("VARIANT", str(a)))
        os.system(
            "pdflatex -synctex=1 -interaction=nonstopmode "+str(a)+".tex"
        )

100%|██████████| 30/30 [00:02<00:00, 10.75it/s]


In [162]:
mergepdf = "pdfunite "
for a in tqdm(range(30)):
    mergepdf += str(a)+".pdf "

100%|██████████| 30/30 [00:00<00:00, 272357.40it/s]


In [164]:
mergepdf+"merged_output.pdf"

'pdfunite 0.pdf 1.pdf 2.pdf 3.pdf 4.pdf 5.pdf 6.pdf 7.pdf 8.pdf 9.pdf 10.pdf 11.pdf 12.pdf 13.pdf 14.pdf 15.pdf 16.pdf 17.pdf 18.pdf 19.pdf 20.pdf 21.pdf 22.pdf 23.pdf 24.pdf 25.pdf 26.pdf 27.pdf 28.pdf 29.pdf merged_output.pdf'

In [159]:
for a in tqdm(range(30)):
    print("pdflatex -synctex=1 -interaction=nonstopmode "+str(a)+".tex")

100%|██████████| 30/30 [00:00<00:00, 34360.76it/s]

pdflatex -synctex=1 -interaction=nonstopmode 0.tex
pdflatex -synctex=1 -interaction=nonstopmode 1.tex
pdflatex -synctex=1 -interaction=nonstopmode 2.tex
pdflatex -synctex=1 -interaction=nonstopmode 3.tex
pdflatex -synctex=1 -interaction=nonstopmode 4.tex
pdflatex -synctex=1 -interaction=nonstopmode 5.tex
pdflatex -synctex=1 -interaction=nonstopmode 6.tex
pdflatex -synctex=1 -interaction=nonstopmode 7.tex
pdflatex -synctex=1 -interaction=nonstopmode 8.tex
pdflatex -synctex=1 -interaction=nonstopmode 9.tex
pdflatex -synctex=1 -interaction=nonstopmode 10.tex
pdflatex -synctex=1 -interaction=nonstopmode 11.tex
pdflatex -synctex=1 -interaction=nonstopmode 12.tex
pdflatex -synctex=1 -interaction=nonstopmode 13.tex
pdflatex -synctex=1 -interaction=nonstopmode 14.tex
pdflatex -synctex=1 -interaction=nonstopmode 15.tex
pdflatex -synctex=1 -interaction=nonstopmode 16.tex
pdflatex -synctex=1 -interaction=nonstopmode 17.tex
pdflatex -synctex=1 -interaction=nonstopmode 18.tex
pdflatex -synctex=1 -i

In [146]:
with open("test_form_result.tex", "w") as oh:
    oh.write(current_form)

In [131]:
anssss

array(['7917224\n', 'SyntaxError: invalid token\n', '7', '769\n'],
      dtype='<U27')

In [115]:
correct_answers[32]

"NameError: name 'q' is not defined\n"

In [116]:
incorrect_answers_1[32]

'7359'

In [117]:
incorrect_answers_2[32]

'72'

In [118]:
incorrect_answers_3[32]

"NameError: name 'g' is not defined\n"